In [ ]:
def preprocessing(input_path):
  all_input_list = sorted(glob.glob(input_path))
  train = pd.DataFrame()
  for datapath in all_input_list:
    data = pd.read_csv(datapath) # 이게 지금 28일치 * 24시
    data['obs_time'] = data.index % 24 # 시간통일
    data = abs(data)
    col_list = data.columns
    for i in range(0,len(col_list)):
      col = col_list[i]    
      if '누적' in col :
        data[col] = data.groupby((data.obs_time == 0).cumsum()).agg(col_list[i-1]).cumsum()
    data.to_csv(datapath,index=False)
    train = pd.concat([train,data])
  return train

In [ ]:
def make_dataset(input_path, target_path):
    train = pd.DataFrame()
    all_input_list = sorted(glob.glob(input_path))
    all_target_list = sorted(glob.glob(target_path))
    for x, y in zip(all_input_list,all_target_list):
        x = pd.read_csv(x)
        y = pd.read_csv(y)
        x = x.iloc[:,1:]
        col_list = x.columns[1:]
        for i in range(0,28) :
            day = x.iloc[24*i:24*i+24]

            day_white = day[day['일간누적백색광량']>=0]['일간누적백색광량'].max()
            day_water = day[day['일간누적분무량']>=0]['일간누적분무량'].max()
            day_light = day[day['일간누적총광량']>=0]['일간누적총광량'].max()
            day_blue = day[day['일간누적청색광량']>=0]['일간누적청색광량'].max()
            day_red = day[day['일간누적적색광량']>=0]['일간누적적색광량'].max()

            time_list = day['obs_time'].unique()
            for col in col_list :
              if col in ['일간누적백색광량','일간누적총광량','일간누적청색광량','일간누적적색광량','일간누적분무량']:
                continue
              else :
                  for t in range(0,len(time_list)-1) :
                      time = time_list[t]
                      ntime = time_list[t+1]
                      value1 = day[day['obs_time']==time][col].iloc[0]
                      value2 = day[day['obs_time']==ntime][col].iloc[0]
                      x[col+str(time)+str("+")+str(ntime)] = (value1 + value2)/2

            x['day_water_weight'] = math.log(day_water*(i+1)+1)
            x['day_white_weight'] = math.log(day_white*(i+1)+1)
            x['day_red_weight'] = math.log(day_red*(i+1)+1)
            x['day_blue_weight'] = math.log(day_blue*(i+1)+1)
            x['day_light_weight'] = math.log(day_light*(i+1)+1)

            x['day_white'] = day_white
            x['day_water'] = day_water
            x['day_red'] = day_red
            x['day_blue'] = day_blue
            x['day_light'] = day_light

            nx = x.iloc[:1,15:]
            ny = y.iloc[i:i+1].reset_index(drop=True)
            xy = pd.merge(nx,ny,left_index=True, right_index=True)
            train = pd.concat([train,xy]).reset_index(drop=True)

    return train

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
train = pd.read_csv('drive/MyDrive/dacon/data/train_weight_log.csv')
test = pd.read_csv('drive/MyDrive/dacon/data/test_weight_log.csv')
X = train.drop(['predicted_weight_g','day_white', 'day_water', 'day_red', 'day_blue','day_light'],axis=1)
y = train['predicted_weight_g']
CAT_model= {}
for f in range(0,27):
     train_idx = [i for i in range(0,784)]
     valid_idx = train_idx[f*28:f*28+28]
     for idx in valid_idx:
       train_idx.remove(idx)
     print(f'===================================={f+1}============================================')
     print(min(valid_idx),max(valid_idx))
     x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]

     CAT = catboost.CatBoostRegressor(task_type='GPU',verbose=0,random_state=404)
     CAT.fit(x_train, y_train)

     y_pred = CAT.predict(x_val)
     mae = mean_squared_error(y_val, y_pred)**0.5
     print(f"{f + 1} Fold MAE = {mae}")
     CAT_model[f] = CAT
     print(f'================================================================================\n\n')